Created: 2020.07.29

Modified: 2020.07.31

# Inference test data with a model

## Axial-2d / bias - mask

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

from fastai.core import parallel

In [4]:
import glob
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
import matplotlib.pyplot as plt

In [6]:
import sys
sys.path.append('functions')

%aimport functions00, functions01
from functions00 import *
from functions01 import *

import functions23 as fun

In [7]:
mk_get_host_info()

**********************************************************************************************************************************
Settings:
	HOST:  mmiv-ml-titan
	PATH_ROOT_DATA:  /data-10tb/shared/skull/train-3d-iso
	PATH_GIT_HUB:  /data-10tb/marek/github_codes/skull-stripping-1/fastai
	PATH_2D: /data-10tb/shared/skull

3D DFs paths (_mk_3D):
	IXI_TEST_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/ixi_test_mk_3d.csv
	TEST_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/test_mk_3d.csv
	TRAIN_VAL_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/train_val_mk_3d.csv
	NFBS_TEST_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/nfbs_test_mk_3d.csv

Error files (pickle):
	ERROR_FILES: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/error_files
**********************************************************************

# Functions

In [4]:
# there are all png files (t1 or t1_biascorr) from selected exam (folder) in selected test data (aibl/hcp)
def assamble_3d_from_2d_slices(pngs_list, mask_pth, save_png_3d=None):
    """
    1 assemble a 3d matrix from set of 2d images
    2 load original mask (eg FSL_outputs)
    3 counts Dice coef
    4 saves assembled 3d matrix to nii file 
    5 save to df: 
    6 msk_cnt, png_cnt, dice
    
    different versions of save function for axial/coronal/saggital
    
    """
    img = open_image(pngs_list[0], convert_mode='L')
    learn.data.single_dl.dataset.tfmargs['size'] = img.shape
    rows, cols =  img.shape[1:]
    sl = len(pngs_list)
    png_img = np.zeros((rows, cols, sl), dtype = 'uint8' )

    name = pngs_list[0]
    if 'axial' in name:
        plane = 'axial'
    elif 'coronal' in name:
        plane = 'coronal'
    elif 'sagittal' in name:
        plane = 'sagittal'
    else:
        print('Sth wrong with file name (assamgle_3d_from_2d_slices)')
        return
    
    # TA CZESC BEDZIE INNA DLA CORONAL I SATTITAL
    # assemble 3d matrix from png's
    for k, png in enumerate(pngs_list):
        img = open_image(png, convert_mode='L')    
        prediction = learn.predict(img)[0]
        if plane == 'axial':
            png_img[:,:,k] = np.squeeze(prediction.data.numpy())
        if plane == 'coronal':
            pass
        if plane == 'sagittal':
            pass
    
    
    # load 3d_mask to compare with (e.g. FSL_outputs)
    msk_nii = nib.load(mask_pth)          
    msk_img = msk_nii.get_fdata()

    # get brain mask voxels (withe voxels)
    msk_cnt = msk_img[msk_img>0].shape[0]
    png_cnt = png_img[png_img>0].shape[0]

    # count dice
    dice = fun.dice_jaccard(msk_img, png_img)    
    
    if save_png_3d:        
        fun.mkSaveImageAsNifti(png_img, msk_nii, save_png_3d)
    return msk_cnt, png_cnt, dice

# Main code

### Learner

In [9]:
# Create a loader to load saved models
# Is any easer and shorter way to load a model???
# It is too complicated.... :(


# dir: axial-2d/coronal/saggita
# df with paht 
# cols ---> t1 OR bias (e.g from csv name)

def create_learner():
    folder_path = PATH_2D / 'axial-2d/'
    
    df = pd.read_csv(folder_path/'bias_mask-test-val-axial-2d.csv')

    codes = np.loadtxt(folder_path/'codes.txt', dtype=str); codes

    src = SegmentationItemList.from_df(df, folder_path, cols='bias_path')\
                                .split_from_df(col='usage')\
                                .label_from_df(cols='mask_full_path', classes=codes)

    size = 128
    bs = 124
    data = (src.transform(get_transforms(), size=size, tfm_y=True) 
            .databunch(bs=bs)
            .normalize(imagenet_stats))
    wd=1e-2
    learn = unet_learner(data, models.resnet34, metrics=dice, wd=wd)
    return learn

In [10]:
# learner
learn = create_learner()

### Global variables, paths

In [14]:
# Tutaj bedzie nazwa pliku testowego.scv!!!!

# One of two test datasets
test1 = 'TEST_3D'
test2 = 'IXI_TEST_3D'
test3 = 'NFBS_TEST_3D'
DATA = test1

MODEL_NR = 3.01

# argumenty funcji
# axial-2d OR coronal-2d OR sagittal-2d : str
plane1 = 'axial-2d'
plane2 = 'coronal-2d'
plane3 = 'sagittal-2d'
PLANE = plane1

# 'T1_iso_' OR 'T1_biascorr_iso_' : str
t1_or_bias1 = 'T1_biascorr_iso_' 
t1_or_bias2 = 'T1_iso_' 
T1_OR_BIAS = t1_or_bias2

In [18]:
##################
### 1. LEARNER ###
##################

# name nn model to specify the test_resulting folder name
# e.g. 3.01
model_name_path = Path(f'{PATH_2D}/{PLANE}/models/{MODEL_NR}_02_train-axial_bias_uf_foc-20_lr-1e3_sp-all')

learn.load(model_name_path);
#learn.destroy()

In [19]:
##########################################################
### II. Data to test on eg: axial-2d/test/AIBL        ###
### Cechy porownania:                                  ###  
###    1. test set: AIBL / HCP                         ###
###    2. plane : axaial-2d / coronal-2d / sagittal-2d ###
###    3. typ obrazu: t1 / t1_biascorr                 ###
##########################################################



############################################
#### III. Prediction (save) folder path  ###
############################################
# folder to store a predicted-assembled 3D nifti images
pred_folder_path = Path (f'{PATH_2D}/pred/{MODEL_NR}/{DATA}/')
pred_folder_path.mkdir(parents=True, exist_ok=True)

# folder to save resulted df's
pred_cvs_folder_path = Path(f'{PATH_GIT_HUB}/5.0/')
pred_cvs_folder_path.mkdir(parents=True, exist_ok=True)

##################################
### IV. TEST (load) folder path ###
##################################
# root folder for patient folders (each contains 2d png images - both t1 and t1_biascorr)
test_folder_path = Path(f'{PATH_2D}/{PLANE}/test/{DATA}')



# # folder with path to reference mask image (iso) and number of voxels classified to a brain mask
# reference_root_folder = Path('/data-10tb/marek/github_codes/skull-stripping-1/fastai/4.0_reference/')
# ref_df = pd.read_csv(f'{reference_root_folder}/{DATA}_ref_FSL_masks_vox_cnt.csv', index_col=None)
# ref_df.head(2)

In [10]:
%%time
##############################################################################################################################################################
### V. assembeld (from 2D slices) 3D_images to compare with e.g. FSL_iso_mask_3d ###
###    1. robie petle for po tych plikach i tworze nazwy plikow do obrobki w punkcie II (na danych testowych), czyli porownoje wybrane (i skonstuowane) 
###        nazwy danych tetowych (duza czesc nazwy plikow png) do obrazow masek iso!!!!
###        
###    2. maska 3d jest jedna ===> z nia bede porownywal obrazy odtworzone ze wszystkich kierunkow (axial/cor/dag) i typow danych (t1 / t1_biascorr)
##############################################################################################################################################################
fsl_masks_folder_path = Path(f'/data-10tb/shared/skull/train-3d-iso/{DATA}')
fsl_masks_full_path_list = sorted(glob.glob(f'{fsl_masks_folder_path}/*_brain_mask_iso.nii.gz'))


k = 0
l = len(fsl_masks_full_path_list)

msk_paths_lst = []
msk_cnt_lst = []
png_cnt_lst = []
dice_lst = []

# TU BEDZIE PETLA FOR DLA WSZYSTKICH TYCH PLIKOW MASEK 3D ISO
# /some/root/folders/exam name [AIBL]/exam name[folder] + image name[nii.gz]
for single_fsl_mask_full_path in fsl_masks_full_path_list:
    #print(1, single_fsl_mask_full_path)

    single_fsl_mask_full_name = single_fsl_mask_full_path.split('/')[-1]
    #print(2, single_fsl_mask_full_name)

    exam_folder_name = single_fsl_mask_full_name.split('_T1')[0]
    #print(3, exam_folder_name)

    cur_exam_full_path = test_folder_path / f'{exam_folder_name}'
    if not cur_exam_full_path.exists(): 
        print(f'Folder {cur_exam_full_path} not exists!!!')

    cur_pngs_full_path_list =  sorted(glob.glob(f'{cur_exam_full_path}/{T1_OR_BIAS}*.png'))
    #print(4, cur_pngs_full_path_list[0])   

    # save name for output png_3d_nii.gz image
    pred_3d_save_name = pred_folder_path / f'{exam_folder_name}_{T1_OR_BIAS}pred_{MODEL_NR}.nii.gz'

    # png_3d_nii.gz image save name
    #nii_pred_3d_save_name = pred_folder_path / str(pred_3d_save_name + '.nii.gz')
    #print(5, nii_pred_3d_save_name)

    # main function ;)
    msk_cnt, png_cnt, dice_c = assamble_3d_from_2d_slices(cur_pngs_full_path_list, single_fsl_mask_full_path, save_png_3d=pred_3d_save_name)
    #print(6, single_fsl_mask_full_name, msk_cnt, png_cnt, dice)

    msk_paths_lst.append(single_fsl_mask_full_path)
    msk_cnt_lst.append(msk_cnt)
    png_cnt_lst.append(png_cnt)
    dice_lst.append(dice_c)
    
    print(f'{k}/{l}\t{exam_folder_name}\t\t\t', end='\r')
    k +=1
    

# df save name     
save_df = f'{MODEL_NR}_{DATA}_{T1_OR_BIAS}{PLANE}_dice.csv'
save_df = pred_cvs_folder_path / save_df

# create df: msk_path, msk_cnt, png_cnt, dice
df = pd.DataFrame({'msk_full_path':msk_paths_lst, 'msk_cnt':msk_cnt_lst, 'png_cnt':png_cnt_lst, 'dice':dice_lst})
df.to_csv(save_df, index=False)

print('Done :)')

Done :)	AIBL_99_MPRAGE_ADNI_confirmed_2008-08-08_11_41_36.0_S78685_I164096.anat							
CPU times: user 1d 6h 44min 55s, sys: 1h 22min 8s, total: 1d 8h 7min 4s
Wall time: 1h 1min 28s


Done :)
